# Simplex algorithm

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from enum import Enum, auto

import numpy as np

from teaching_optimization.linear_optimization import LinearOptimization
from teaching_optimization.linear_constraints import (
    draw_polyhedron_standard_form,
    LabeledPoint,
)


In this lab, you will **implement the simplex algorithm** step by step and see how it moves
from one basic feasible solution to another along the edges of the feasible polyhedron.
You will choose an entering variable using reduced costs, compute the basic direction,
find the maximum feasible step (leaving variable), and update the basis—repeating until
optimality, unboundedness, or infeasibility is detected. The goal is to connect the algebra
(bases, reduced costs, pivoting) with the geometry (vertices, edges, level lines) so that
each iteration of simplex becomes intuitive and its stopping conditions make sense.

Consider the following linear optimization problem
$$
\max 4x_1 - 3x_2
$$
subject to
\begin{align*}
2x_1  +  x_2 &\leq  6,\\
x_1 - x_2    &\leq  2,\\
x_{1}  ,  x_{2}  &\geq  0. \\
\end{align*}

The objective of this exercise is to implement the simplex algorithm to find the optimal solution.
Verify that the point (0, 0) is a basic feasible solution. If so, start the algorithm from this point.


First,  write the problem in standard form.

In [ ]:
standard_a = ...
standard_b = ...
standard_c = ...


Initialize the optimization problem.

In [ ]:
the_problem = LinearOptimization(
    objective=standard_c, constraint_matrix=standard_a, right_hand_side=standard_b
)


Read the documentation.

In [ ]:
help(LinearOptimization)


Starting point. Specify the indices of the basic variables that correspond to the point (0, 0).

In [ ]:


the_basic_indices = ...
print(f'Basic variables: {the_basic_indices}')


Check that it is feasible.

In [ ]:
the_problem.basic_indices = the_basic_indices
the_basic_solution = the_problem.basic_solution
if the_basic_solution is not None:
    print(f'Starting point: {the_basic_solution}')
else:
    raise ValueError('Invalid starting basis')
if the_problem.is_basic_solution_feasible:
    print(f'Starting point is feasible')
else:
    raise ValueError('Infeasible starting basic solution')



Write a function that identifies a non-basic index to enter the basis

In [ ]:
def index_entering_basis(
    optimization_problem: LinearOptimization, basic_indices: list[int]
) -> int | None:
    """
    Function that identifies a non-basic index to enter the basis, or detect optimality
    :param optimization_problem: object representing the problem.
    :param basic_indices: current list of basic indices.
    :return: a non-basic index corresponding to a negative reduced cost, or None if optimal.
    """
    optimization_problem.basic_indices = basic_indices
    for non_basic_index in optimization_problem.non_basic_indices:
        reduced_cost = ...











We test the function. Expected result: Entering variable: 0

In [ ]:
the_entering_index = index_entering_basis(
    optimization_problem=the_problem, basic_indices=the_basic_indices
)
print(f'Entering variable: {the_entering_index}')


Verify the reduced cost for this variable. It must be negative.

In [ ]:
print(f'Reduced cost: {the_problem.reduced_cost(variable_index=the_entering_index)}')



Write a function that calculates the longest step along a direction.

In [ ]:
def longest_step(
    current_point: np.ndarray, direction: np.ndarray
) -> tuple[float, int | None]:
    """
    Function that calculates the longest step along a direction so that all entries stay non-negative.
    :param current_point: current point.
    :param direction: direction.
    :return: the step, as well as the index of one variable that becomes zero, or None of the direction is unbounded.
    """
    ...







    return min_value, min_index



We test the function. Expected result: Longest step: 1.0. Activated constraint: 2.

In [ ]:
the_point = np.array([3, 2, 1])
the_direction = np.array([-1, 1, -1])
the_alpha, the_index = longest_step(current_point=the_point, direction=the_direction)
print(f'Longest step: {the_alpha}. Activated constraint: {the_index}')


We test again with an unbounded direction. Expected result: Longest step: inf. Activated constraint: None.

In [ ]:
the_point = np.array([3, 2, 1])
the_direction = np.array([1, 1, 1])
the_alpha, the_index = longest_step(current_point=the_point, direction=the_direction)
print(f'Longest step: {the_alpha}. Activated constraint: {the_index}')



Write a function that identifies a basic index to leave the basis.

In [ ]:
def index_leaving_basis(
    optimization_problem: LinearOptimization,
    basic_indices: list[int],
    index_entering: int,
) -> int | None:
    """function that identifies a basic index to leave the basis, or identify an unbounded problem.

    :param optimization_problem: object representing the problem.
    :param basic_indices: current list of basic indices.
    :param index_entering: non-basic index entering the basis
    :return: index of the variable leaving the basis, or None if unbounded
    """
    optimization_problem.basic_indices = basic_indices
    basic_part_basic_solution = (
        optimization_problem.basic_part_basic_solution
    ...










    if index is None:
        # The problem is unbounded along the direction.
        return None
    return basic_indices[index]



We test the function. Expected result: Exiting variable: 3.

In [ ]:
the_exiting_index = index_leaving_basis(
    optimization_problem=the_problem,
    basic_indices=the_basic_indices,
    index_entering=the_entering_index,
)

print(f'Exiting variable: {the_exiting_index}')



We define a list of possible interruptions of the algorithm

In [ ]:
class CauseInterruptionIterations(Enum):
    OPTIMAL = auto()
    UNBOUNDED = auto()
    INFEASIBLE = auto()

    def __str__(self) -> str:
        messages = {
            self.OPTIMAL: 'Optimal basis found.',
            self.UNBOUNDED: 'Optimization problem is unbounded.',
            self.INFEASIBLE: 'Optimization problem is infeasible.',
        }
        return messages[self]



Write a function that performs one iteration of the simplex algorithm.

In [ ]:
def simplex_iteration(
    optimization_problem: LinearOptimization, basic_indices: list[int]
) -> tuple[list[int] | None, CauseInterruptionIterations | None]:
    """
    Performs one iteration of the simplex algorithm.
    :param optimization_problem: object representing the problem.
    :param basic_indices: current list of basic indices.
    :return: new list of basic indices, if successful. If None, a message explaining the reason.
    """



    if ...:
        # Optimal solution found.
        return None, CauseInterruptionIterations.OPTIMAL






    if ...:
        # Problem is unbounded.
        return None, CauseInterruptionIterations.UNBOUNDED




    return new_basic_indices, None



We apply the functions on the example above.
Starting basic indices: 2 and 3.

In [ ]:
the_basic_indices = [2, 3]
print(f'Basic variables: {the_basic_indices}')
the_problem.basic_indices = the_basic_indices
the_basic_solution = the_problem.basic_solution


Draw the polyhedron and the current solution.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    points=[LabeledPoint(coordinates=the_basic_solution, label='It. 0', shift=0.1)],
)


First iteration

In [ ]:
basic_indices_1, interruption = simplex_iteration(
    optimization_problem=the_problem, basic_indices=the_basic_indices
)
if basic_indices_1 is None:
    print(interruption)

print(f'Basic variables: {basic_indices_1}')
the_problem.basic_indices = basic_indices_1


Draw the polyhedron and the current solution.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    points=[
        LabeledPoint(coordinates=the_problem.basic_solution, label='It. 1', shift=0.1)
    ],
)


Second iteration

In [ ]:
basic_indices_2, interruption = simplex_iteration(
    optimization_problem=the_problem, basic_indices=basic_indices_1
)
if basic_indices_2 is None:
    print(interruption)

print(f'Basic variables: {basic_indices_2}')
the_problem.basic_indices = basic_indices_2


Draw the polyhedron and the current solution.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    points=[
        LabeledPoint(coordinates=the_problem.basic_solution, label='It. 2', shift=0.1)
    ],
)


Third iteration

In [ ]:
basic_indices_3, interruption = simplex_iteration(
    optimization_problem=the_problem, basic_indices=basic_indices_2
)
if basic_indices_3 is None:
    print(interruption)


Draw the polyhedron and the current solution.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    objective_coefficients=standard_c,
    level_lines=[-26.0 / 3.0, 0, 26.0 / 3.0],
    points=[
        LabeledPoint(coordinates=the_problem.basic_solution, label='Optimum', shift=0.1)
    ],
)